COMP 215 - LAB 3 Classes (NEO)
----------------
#### Name: Falak Jain
#### Date: 29 January 2024

***For these lab exercises, I took help of AI and some of my friends for a better concept clarity... I did not merely copy paste codes from anywhere, but yes I took help to understand the approach and syntaxes for my exercises... I really hope that is acceptable :) ...***

This lab exercise introduces `class` as a means of organizing related data and functions.

**Building on new concepts from lab 2**:
  * a `record` is a related collection of data, with fields for each data value
  * an `API` is an "Application Programmers Interface" defining how a programmer interacts with a system.
  * *f-string* simplifies string formatting operations

**New Python Concepts**:
  * the `class` keyword allows you define a new data `type`, with a set of operations on that data.
  * a `dataclass` simplifies class definition for classes that primarily encapsulate a data structure.

As usual, the first code cell simply imports all the modules we'll be using...

In [ ]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll continue working with [Near Earth Object](https://cneos.jpl.nasa.gov/) data
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

Here's a brief review from Lab 2 on how to use it...

### Review: making a query

Here's a query that gets the record for a single NEO that recently passed by.

In [ ]:
API_KEY = '1BlH27r67PFVu8MP7jAtDOaKswDtU6xAV1LzdiCr'  # substitute your API key here

def get_neos(start_date):
    """ Return a list of NEO for the week starting at start_date """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&api_key={API_KEY}'
    # Fetch last week's NEO feed
    response = requests.request("GET", url, headers={}, data={})
    data = json.loads(response.text)
    print(f'{data["element_count"]} Near Earth Objects found for week of {start_date}')
    return [neo for dated_records in data['near_earth_objects'].values() for neo in dated_records ]

def get_neo(id):
    """ Return a NEO record for the given id """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
    response = requests.request("GET", url, headers={}, data={})
    return json.loads(response.text)

week_start = '2023-01-15'
neos = get_neos(week_start)
assert len(neos) > 0, f'Oh oh!  No NEOs found for {week_start}'
neo = get_neo(neos[-1]['id'])  # get the very latest NEO
pprint(neo)

## Exercise 1:  Define an Asteroid class

Define a simple Asteroid class with some basic state variables for a single NEO.  Your Asteroid class should define at least 4 "state variables:”

    id
    name
    estimated_diameter (m)
    is_potentially_hazardous (bool)

Operations must include:
* `__init__(self, ...)` method to initialize a new Asteroid object with specific data values
* `__str__(self)`, and `__repr__(self)` methods that return nicely formatted string representations of the object.
  
OR...
use a `@dataclass` and it will supply most of that boilerplate code for you!

Write a little code to test your new class.

In [ ]:
# Ex.1 your code here
class Neo_asteroid:
  def __init__(self , id:int , name:str , estimated_diameter:float , is_potentially_hazardous:bool):
    self.id = id
    self.name = name
    self.estimated_diameter = estimated_diameter
    self.is_potentially_hazardous = is_potentially_hazardous

  def __str__(self):
    return f"Id : {self.id} , Name : {self.name} , Estimated Diameter : {self.estimated_diameter} , Hazardous : {self.is_potentially_hazardous}"

asteroid = Neo_asteroid(1 , 'ABC' , 5.0 , False)
type(asteroid)

__main__.Neo_asteroid

## Exercise 2: Factory method: Asteriod.from_NEO

We want to be able to construct Asteroid objects easily from the record returned from the NEO API.  

Add an "object factory" method to your class...   

    @classmethod
    def from_NEO(cls, neo_id):
        ...

that takes the id for a single NEO, fetches the NEO record from API, constructs and returns an Asteroid object representing that NEO.
This kind of method is called a “Factory” because it constructs an object from raw materials.

Write a little code to test your new class.

In [ ]:
# Ex. 2 your code here

class Neo_asteroid:
  def __init__(self , id:int , name:str , estimated_diameter:float , is_potentially_hazardous:bool):
    self.id = id
    self.name = name
    self.estimated_diameter = estimated_diameter
    self.is_potentially_hazardous = is_potentially_hazardous

  def __str__(self):
    return f"Id : {self.id} , Name : {self.name} , Estimated Diameter : {self.estimated_diameter} , Hazardous : {self.is_potentially_hazardous}"

  @classmethod
  def from_NEO(cls , neo_id):
    record = get_neo(neo_id)
    id = record['id']
    name = record['name']
    estimated_diameter = record['estimated_diameter']['meters']['estimated_diameter_max']
    is_potentially_hazardous = record['is_potentially_hazardous_asteroid']

    return cls(id , name , estimated_diameter , is_potentially_hazardous)

test_case = neos[-1]['id']    #latest neo (from review query)
asteroid2 = Neo_asteroid.from_NEO(test_case)
print(asteroid2)

Id : 54339874 , Name : (2023 BM1) , Estimated Diameter : 63.3928064522 , Hazardous : False


## Exercise 3: Define a CloseApproach class

Each NEO record comes with a list of `close_approach_data`, where each record in this list represents a single “close approach” to another orbiting body.
* Develop a class named `CloseApproach` to represent a single close approach record.
* State variables are

        neo (refrence to related NEO object)
        orbiting body (str)
        approach date (datetime object!)
        miss distance (float - choose units, document it, and be consistent!)
        relative velocity (ditto)
  
Define a "Factory" class method to construct a `CloseApproach` object from one close approach data record (a dictionary object).   
This method takes a `neo` object as input for the NEO to which the close approach data belongs.
Remember to parse the date/time string into a datetime object.

In [ ]:
# Ex. 3 your code here

class CloseApproach:
  def __init__(self , neo , orbiting_body , approach_datetime , miss_distance , relative_velocity):
    self.neo = neo
    self.orbiting_body = orbiting_body
    self.approach_datetime = approach_datetime
    self.miss_distance = miss_distance
    self.relative_velocity = relative_velocity

  def __str__(self):
    return f"NEO OBJECT : {self.neo} , Orbiting Body : {self.orbiting_body} , Approach Date : {self.approach_datetime} , Missed Distance : {self.miss_distance}km , Relative Velocity : {self.miss_distance}km/h"

  @classmethod
  def from_close_approach_data_record(cls , neo , close_approach_data):
    orbiting_body = close_approach_data['orbiting_body']
    approach_datetime_asStr = str(close_approach_data['close_approach_date_full'])
    miss_distance = float(close_approach_data['miss_distance']['kilometers'])
    relative_velocity = float(close_approach_data['relative_velocity']['kilometers_per_hour'])
    approach_datetime = datetime.datetime.strptime(approach_datetime_asStr , '%Y-%m-%d %H:%M')

    return cls(neo , orbiting_body , approach_datetime , miss_distance , relative_velocity)

test_case_neo = Neo_asteroid.from_NEO(test_case)    #latest neo (from review query)
test_case_close_approach_data = {'orbiting_body':'Earth' ,
                   'close_approach_date_full':'2104-07-20 13:54' ,
                   'miss_distance':{'kilometers':'290' ,
                                    'lunar':'150'} ,
                   'relative_velocity': {'kilometers_per_hour': '63429.8406460211',
                                         'kilometers_per_second': '17.6194001795',}
                   }

close_approachh = CloseApproach.from_close_approach_data_record(test_case_neo , test_case_close_approach_data)
print(close_approachh)


NEO OBJECT : Id : 54339874 , Name : (2023 BM1) , Estimated Diameter : 63.3928064522 , Hazardous : False , Orbiting Body : Earth , Approach Date : 2104-07-20 13:54:00 , Missed Distance : 290.0km , Relative Velocity : 290.0km/h


## Exercise 4: Add list of CloseApproach objects to the Asteroid

Every `Asteroid` should have a list of “close approaches”.
But there is a catch-22 here because we need the `Asteroid` to construct the `CloseApproach` objects.
Add an instance variable to your Asteroid class with a default value of an empty list:

    ...
    close_approaches:list = []
    ...
      
In `from_NEO` factory, use a list comprehension to build the list of `CloseApproach` objects for the Asteroid instance, and then set the instance's `close_approaches` variable before returning it.  
Setting the value of an object's instance variables from outside the class is generally frowned upon - this is why we make the factory a method of the class itself!

Now add a method to `Asteroid` to return the `nearest_miss` `CloseApproach` object for the asteroic:

    def nearest_miss(self):
        ...

Extend your test code to demonstrate these new features.

In [ ]:
# Ex. 4 your code here

# copied code from Ex. 2 here

class Neo_asteroid:
  def __init__(self , id:int , name:str , estimated_diameter:float , is_potentially_hazardous:bool):
    self.id = id
    self.name = name
    self.estimated_diameter = estimated_diameter
    self.is_potentially_hazardous = is_potentially_hazardous
    close_approaches:list = []

  def __str__(self):
    return f"Id : {self.id} , Name : {self.name} , Estimated Diameter : {self.estimated_diameter} , Hazardous : {self.is_potentially_hazardous}"

  @classmethod
  def from_NEO(cls , neo_id):
    record = get_neo(neo_id)
    id = record['id']
    name = record['name']
    estimated_diameter = record['estimated_diameter']['meters']['estimated_diameter_max']
    is_potentially_hazardous = record['is_potentially_hazardous_asteroid']

    asteroid_instance = cls(id , name , estimated_diameter , is_potentially_hazardous)
    asteroid_instance.close_approaches = [CloseApproach.from_close_approach_data_record(asteroid_instance , data) for data in record['close_approach_data']]
    return asteroid_instance

  def nearest_miss(self):
    if not self.close_approaches:
      return None

    nearest_miss_distance = min(self.close_approaches , key=lambda approach: approach.miss_distance)
    return nearest_miss_distance

test_case = neos[-1]['id']    #latest neo (from review query)
asteroid_neo_2 = Neo_asteroid.from_NEO(test_case)
print(asteroid_neo_2)

ValueError: time data '1949-Jul-04 22:19' does not match format '%Y-%m-%d %H:%M'

## Challenge - Take your skills to the next level...
### Exercise 5: add one additional analysis

 With these data structures in place, we can now start answering all kinds of interesting questions about a single Asteroid or a set of Asteroids.  
Here’s a couple ideas to try:

* add a method to the Asteroid class, `closest_earth_approach`, that returns the CloseApproach object that represents the closest approach the Asteroid makes to Earth.

* write a **function** named `most_dangerous_approach`, that takes a date range and returns a single “potentially hazardous” Asteroid object that makes the closest approach to Earth in within that range.  Your algorithm will ultimately need to:
    * grab the list of NEO’s for the given date range;
    * use a list comprehension to build the list of Asteroid objects for the NEO’s returned
    * use a list comprehension to filter  potentially hazardous Asteroids only;
    * use a list comprehension to map each Asteroid to its  closest_earth_approach
    * apply Python’s min function to identify the Asteroid with the closest_earth_approach

You may want to decompose some of these steps into smaller functions.
* add a method to the Asteroid class, estimated_mass, that computes an estimate of the Asteroid’s mass based on its diameter.  This is a model – state your assumptions.
* add a method to the CloseApproach class, impact_force,  that estimates the force of impact if the Asteroid hit the orbiting object.  Again, this is a model, state your assumptions.

In [ ]:
# Ex. 5 (challenge) your code here